## Virginia Counties
#### Exploratory Data Analysis

https://www.kaggle.com/muonneutrino/us-census-demographic-data/download

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
import plotly as py
import plotly.graph_objs as go
# from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [4]:
# read in the dataset
counties = pd.read_csv('https://raw.githubusercontent.com/austinlasseter/dash-virginia-counties/master/resources/acs2017_county_data.csv')
counties.head()

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,...,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,...,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,...,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,...,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,...,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


In [5]:
usda=pd.read_excel('https://github.com/austinlasseter/dash-virginia-counties/raw/master/resources/ruralurbancodes2013.xls')
usda.sample(3)

,FIPS,State,County_Name,Population_2010,RUCC_2013,Description
2162,40063,OK,Hughes County,14003,7,"Nonmetro - Urban population of 2,500 to 19,999..."
2603,48161,TX,Freestone County,19816,7,"Nonmetro - Urban population of 2,500 to 19,999..."
2790,49027,UT,Millard County,12503,7,"Nonmetro - Urban population of 2,500 to 19,999..."


In [22]:
fulldf=pd.merge(counties, usda, left_on='CountyId', right_on='FIPS', how='left')

In [41]:
fulldf.head()

,CountyId,State_x,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,PublicWork,SelfEmployed,FamilyWork,Unemployment,FIPS,State_y,County_Name,Population_2010,RUCC_2013,Description
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,...,20.2,5.6,0.1,5.2,1001.0,AL,Autauga County,54571.0,2.0,"Metro - Counties in metro areas of 250,000 to ..."
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,...,12.9,6.3,0.1,5.5,1003.0,AL,Baldwin County,182265.0,3.0,Metro - Counties in metro areas of fewer than ...
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,...,19.1,6.5,0.3,12.4,1005.0,AL,Barbour County,27457.0,6.0,"Nonmetro - Urban population of 2,500 to 19,999..."
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,...,17.4,6.3,0.3,8.2,1007.0,AL,Bibb County,22915.0,1.0,Metro - Counties in metro areas of 1 million p...
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,...,11.9,4.0,0.1,4.9,1009.0,AL,Blount County,57322.0,1.0,Metro - Counties in metro areas of 1 million p...


In [58]:
fulldf.sample(1)

,CountyId,State_x,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,PublicWork,SelfEmployed,FamilyWork,Unemployment,FIPS,State_y,County_Name,Population_2010,RUCC_2013,Description
2484,47113,Tennessee,Madison County,97887,46449,51438,3.7,56.5,37.1,0.0,...,13.2,5.6,0.3,6.2,47113.0,TN,Madison County,98294.0,3.0,Metro - Counties in metro areas of fewer than ...


In [61]:
#statedf = fulldf.groupby('State_y', as_index=False)[['State_y', 'TotalPop']].mean()
#statedf = fulldf.groupby('State_y')[['State_y', 'TotalPop']].mean().reset_index()
statedf = fulldf.groupby(['State_y', 'State_x'])[['TotalPop']].mean().reset_index()
statedf.head()

,State_y,State_x,TotalPop
0,AK,Alaska,26087.000000
1,AL,Alabama,72399.567164
2,AR,Arkansas,39705.920000
3,AZ,Arizona,453996.400000
4,CA,California,672118.051724


In [70]:
statecode = 'AL'
singlestatedf = statedf[statedf['State_y'] == statecode]

In [74]:

data = [dict(type='choropleth', 
    colorscale='Blues', 
    z=singlestatedf['TotalPop'], 
    locations=singlestatedf['State_y'], 
    locationmode='USA-states', 
    text = singlestatedf['State_y'], 
    colorbar=dict(title="millions"))]

data

[{'type': 'choropleth',
  'colorscale': 'Blues',
  'z': 1    72399.567164
  Name: TotalPop, dtype: float64,
  'locations': 1    AL
  Name: State_y, dtype: object,
  'locationmode': 'USA-states',
  'text': 1    AL
  Name: State_y, dtype: object,
  'colorbar': {'title': 'millions'}}]

In [72]:
layout = dict(title='Total Population', 
    geo = dict(scope='usa', projection=dict(type='albers usa'), showlakes = True, lakecolor = 'rgb(66,165,245)'))

layout

{'title': 'Total Population',
 'geo': {'scope': 'usa',
  'projection': {'type': 'albers usa'},
  'showlakes': True,
  'lakecolor': 'rgb(66,165,245)'}}

In [ ]:
pip install --upgrade nbformat

In [73]:
fig = go.Figure(data=data, layout=layout)
fig.show()
#iplot(fig)

In [ ]:
counties.info()
counties["State"]=='Virginia'

In [ ]:
# restrict to Virginia
va = counties.loc[counties['State']=='Virginia']
print(counties.shape)
print(va.shape)

In [ ]:
# show a list of variables
va.info()

#### Data Dictionary
| Varname | Data | 
|:---:|:---|
| CensusTract | Census tract ID |
| State | State, DC, or Puerto Rico |
|County | County or county equivalent | 
| Total | PopTotal population | 
| Men | Number of men |
|Women|Number of women |
|Hispanic|% of population that is Hispanic/Latino|
|White|% of population that is white|
|Black|% of population that is black|
|Native|% of population that is Native American or Native Alaskan|
|Asian|% of population that is Asian|
|Pacific|% of population that is Native Hawaiian or Pacific Islander|
|Citizen|Number of citizens|
|Income|Median household income |
|IncomeErr|Median household income error|
|IncomePerCap|Income per capita |
|IncomePerCapErr|Income per capita error |
|Poverty|% under poverty level|
|ChildPoverty|% of children under poverty level|
|Professional|% employed in management, business, science, and arts|
|Service|% employed in service jobs|
|Office|% employed in sales and office jobs|
|Construction|% employed in natural resources, construction, and maintenance|
|Production|% employed in production, transportation, and material movement|
|Drive|% commuting alone in a car, van, or truck|
|Carpool|% carpooling in a car, van, or truck|
|Transit|% commuting on public transportation|
|Walk|% walking to work|
|OtherTransp|% commuting via other means|
|WorkAtHome|% working at home|
|MeanCommute|Mean commute time (minutes)|
|Employed|Number of employed (16+)|
|PrivateWork|% employed in private industry|
|PublicWork|% employed in public jobs|
|SelfEmployed|% self-employed|
|FamilyWork|% in unpaid family work|
|Unemployment|Unemployment rate (%)|

In [ ]:
va.columns

### Rural or Urban?
https://www.ers.usda.gov/data-products/rural-urban-continuum-codes/

In [ ]:
# get the USDA county rural-urban codes
usda=pd.read_excel('https://github.com/austinlasseter/dash-virginia-counties/raw/master/resources/ruralurbancodes2013.xls')
usda.sample(3)

In [ ]:
# what are the codes?
usda.groupby('RUCC_2013')[['RUCC_2013','Description']].max()

In [ ]:
# merge with VA data
va2=pd.merge(va, usda, left_on='CountyId', right_on='FIPS', how='left')
print(va.shape)
print(va2.shape)

In [ ]:
# what are the counts?
va2['RUCC_2013'].value_counts()

In [ ]:
# simplify the rural-urban indicator
va2['metro']=va2['RUCC_2013'].map({1:'urban', 
                                   2:'suburban',
                                   3:'suburban',
                                   4:'town',
                                   5:'town',
                                   6:'town',
                                   7:'rural',
                                   8:'rural',
                                   9:'rural'})
# sort as categories
va2['metro'] = pd.Categorical(va2['metro'], ['urban', 'suburban', 'town', 'rural'])
va2['metro'].value_counts()

In [ ]:
# are there differences in commuting and transit patterns?
va2.groupby('metro')[['MeanCommute', 'Drive','Carpool','Transit']].mean().sort_index()

In [ ]:
# are there differences in income and poverty?
va2.groupby('metro')[['Income', 'IncomePerCap','Unemployment', 'Poverty']].mean().sort_index()

In [ ]:
# are there differences in race-ethnicity?
va2.groupby('metro')[['White','Black','Hispanic','Asian','Native']].mean().sort_index()

In [ ]:
# are there differences in industry?
va2.groupby('metro')[['Professional','Service','Office','Construction','Production']].mean().sort_index()

In [ ]:
# Differences in sector?
va2.groupby('metro')[['PrivateWork','PublicWork']].mean().sort_index()

In [ ]:
# display all counties
# !pip install plotly-geo
# import plotly.figure_factory as ff

# values = va2['Income']

# fig = ff.create_choropleth(fips=va2['FIPS'], 
#                            values=values, 
#                            scope=['VA'], 
#                            county_outline={'color': 'rgb(255,255,255)', 'width': 0.5})
# iplot(fig)

In [ ]:
# export
# va2.to_pickle('va-stats.pkl')

In [ ]:
va2['FIPS'].dtype
va2['fips']=va2['FIPS'].astype(str)

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

fig = go.Figure(go.Choroplethmapbox(geojson=counties, 
                                    locations=va2['fips'], 
                                    z=va2['Income'],
                                    colorscale='Electric', 
                                    text=va2['County'],
                                    zmin=0, 
                                    zmax=100000,
                                    marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=5.8, mapbox_center = {"lat": 38.0293, "lon": -79.4428})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
iplot(fig)
# https://community.plot.ly/t/what-colorscales-are-available-in-plotly-and-which-are-the-default/2079